In [21]:
# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os

import dotenv
from dotenv import load_dotenv
load_dotenv()
# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd

# Para poner barras progreso
# -----------------------------------------------------------------------
from tqdm import tqdm

# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

from datetime import datetime

import psycopg2
from psycopg2 import sql
# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [22]:
df_economico=pd.read_csv("../data/datos_agregados/datos_economicos_agregados.csv", index_col=0)
df_poblacional=pd.read_csv("../data/datos_agregados/datos_poblacionales_agregados.csv",index_col=0)
df_todo_demanda=pd.read_csv("../data/datos_agregados/datos_demanda_agregados.csv",index_col=0)
df_todo_oferta=pd.read_csv("../data/datos_agregados/datos_generacion_agregados.csv",index_col=0)

Vamos a hacernos una idea de como son estos dataframes

In [23]:
df_todo_oferta.head(2)


,value,percentage,datetime,type,cod_ccaa,ccaa
0,25288.605,0.029517,2018,Hidráulica,4,Andalucía
1,33933.350,0.030779,2019,Hidráulica,4,Andalucía


In [24]:
df_todo_oferta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2909 entries, 0 to 2908
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       2909 non-null   float64
 1   percentage  2909 non-null   float64
 2   datetime    2909 non-null   int64  
 3   type        2909 non-null   object 
 4   cod_ccaa    2909 non-null   int64  
 5   ccaa        2909 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 159.1+ KB


In [25]:
df_todo_oferta.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2909 entries, 0 to 2908
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       2909 non-null   float64
 1   percentage  2909 non-null   float64
 2   datetime    2909 non-null   int64  
 3   type        2909 non-null   object 
 4   cod_ccaa    2909 non-null   int64  
 5   ccaa        2909 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 159.1+ KB


In [26]:
df_todo_demanda.head(2)


,value,percentage,datetime,cod_ccaa,ccaa
0,3668893.328,1,2018,4,Andalucía
1,3152278.091,1,2019,4,Andalucía


In [27]:
df_todo_demanda.info()


<class 'pandas.core.frame.DataFrame'>
Index: 684 entries, 0 to 11
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       684 non-null    float64
 1   percentage  684 non-null    int64  
 2   datetime    684 non-null    int64  
 3   cod_ccaa    684 non-null    int64  
 4   ccaa        684 non-null    object 
dtypes: float64(1), int64(3), object(1)
memory usage: 32.1+ KB


In [28]:
df_poblacional.head(2)


,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,02 Albacete,0-15 años,Españoles,Hombres,2021,27.304
1,02 Albacete,0-15 años,Españoles,Hombres,2020,27.979


In [29]:
df_poblacional.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1872 entries, 0 to 1871
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Provincias               1872 non-null   object
 1   Edad (3 grupos de edad)  1872 non-null   object
 2   Españoles/Extranjeros    1872 non-null   object
 3   Sexo                     1872 non-null   object
 4   Año                      1872 non-null   int64 
 5   Total                    1872 non-null   object
dtypes: int64(1), object(5)
memory usage: 102.4+ KB


In [30]:
df_economico.head(3)


,Provincias,Ramas de actividad,periodo,Total
0,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2021,8.663.814
1,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2020,7.855.556
2,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2019,8.475.070


In [31]:
df_economico.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1716 entries, 0 to 1715
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Provincias          1716 non-null   object
 1   Ramas de actividad  1716 non-null   object
 2   periodo             1716 non-null   int64 
 3   Total               1716 non-null   object
dtypes: int64(1), object(3)
memory usage: 67.0+ KB


### Creación de tablas

Empezamos por aquellas sin foreign key

In [32]:
# Función para crear un id
def crear_id(df_antiguo,nombre_col_antigua,nombre_col_nueva,nombre_col_id_nuevo):
    anio= pd.DataFrame()
    anio[nombre_col_nueva]=df_antiguo[nombre_col_antigua].unique()
    anio.index=anio.index+1
    df_nuevo=anio.reset_index().rename(columns={'index': nombre_col_id_nuevo})
    return df_nuevo

In [33]:
df_anio=crear_id(df_economico,"periodo","anio","id_anio")

In [34]:
df_anio

,id_anio,anio
0,1,2021
1,2,2020
2,3,2019
